In [1]:
import argparse
import os
import sys
import json
import shutil
import pathlib
import random
from itertools import islice
import time
from copy import deepcopy
import math
from pprint import pprint
from qj_global import qj
import logging

try:
    from comet_ml import Experiment
    COMET_AVAIL = True
except:
    COMET_AVAIL = False

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

from ignite.engine import Engine, Events
from ignite.handlers import Checkpoint, DiskSaver, Timer, TerminateOnNan, TimeLimit
from ignite.metrics import RunningAverage
from ignite.contrib.metrics import GpuInfo
from ignite.utils import setup_logger

# adding the folder containing the folder `disentanglement_via_mechanism_sparsity` to sys.path
# sys.path.insert(0, str(pathlib.Path(__file__).parent.parent))
sys.path.insert(0, str('/Users/vitoria/Documents/GitHub/'))
from disentanglement_via_mechanism_sparsity.universal_logger.logger import UniversalLogger
from disentanglement_via_mechanism_sparsity.metrics import MyMetrics, linear_regression_metric, mean_corr_coef, edge_errors
from disentanglement_via_mechanism_sparsity.plot import plot_01_matrix
from disentanglement_via_mechanism_sparsity.data.synthetic import get_ToyManifoldDatasets
from disentanglement_via_mechanism_sparsity.model.ilcm_vae import ILCM_VAE
from disentanglement_via_mechanism_sparsity.model.latent_models_vae import FCGaussianLatentModel


In [2]:
manifold = "nn"
transition_model = "simplest_discrete"

In [3]:
datasets = get_ToyManifoldDatasets(manifold, transition_model, split=(0.8, 0.1, 0.1),
                                       z_dim=2, x_dim=6, num_samples=100,
                                       no_norm=True, discrete=True)

/Users/vitoria/Documents/GitHub/disentanglement_via_mechanism_sparsity/data/synthetic.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z = torch.tensor(z, dtype=dtype).to(device)


In [4]:
datasets[-3].__getitem__(5)

(tensor([[[ 0.0039,  0.0235, -0.0282, -0.0366, -0.0108,  0.0056],
          [ 0.0258,  0.0190,  0.0213,  0.0121,  0.0041, -0.0124]]]),
 tensor([3., 2.]),
 tensor([0]),
 True,
 array([], shape=(0, 2, 4), dtype=float64),
 tensor([[[2., 0.],
          [2., 3.]]], dtype=torch.float64),
 tensor([[[ 0.0039,  0.0235, -0.0282, -0.0366, -0.0108,  0.0056],
          [ 0.0258,  0.0190,  0.0213,  0.0121,  0.0041, -0.0124]]]))

In [ ]:
datasets[-3].__getitem__(5)[]

In [5]:
datasets[-3].__getitem__(6)

(tensor([[[ 0.0019,  0.0118, -0.0141, -0.0183, -0.0054,  0.0028],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]]),
 tensor([3., 2.]),
 tensor([0]),
 True,
 array([], shape=(0, 2, 4), dtype=float64),
 tensor([[[1., 0.],
          [0., 0.]]], dtype=torch.float64),
 tensor([[[ 0.0019,  0.0118, -0.0141, -0.0183, -0.0054,  0.0028],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]]))

In [3]:
import numpy as np

In [4]:
num_samples=100
z_dim=2
k=3

In [5]:
a = np.random.choice(k,(num_samples, z_dim))

In [6]:
a.shape

(100, 2)

NameError: name 'z' is not defined

In [7]:
G = torch.eye(2)

In [8]:
G

tensor([[1., 0.],
        [0., 1.]])

In [9]:
G[a[1,0]]

tensor([1., 0.])

In [36]:
a[0]

array([2, 1])

In [37]:
a[0,0]

2

In [38]:
G = np.eye(2)

In [39]:
G

array([[1., 0.],
       [0., 1.]])

In [40]:
len(a)

100

In [37]:
aa = torch.Tensor(a)

In [38]:
aa.dtype

torch.float32

In [39]:
aa

tensor([[0., 1.],
        [0., 1.],
        [2., 1.],
        [2., 1.],
        [1., 2.],
        [1., 0.],
        [0., 1.],
        [2., 2.],
        [2., 1.],
        [1., 0.],
        [0., 1.],
        [0., 0.],
        [2., 2.],
        [0., 1.],
        [2., 2.],
        [1., 2.],
        [2., 1.],
        [2., 2.],
        [1., 1.],
        [1., 0.],
        [1., 0.],
        [1., 2.],
        [0., 1.],
        [2., 1.],
        [0., 0.],
        [0., 2.],
        [1., 1.],
        [2., 1.],
        [0., 0.],
        [0., 1.],
        [2., 1.],
        [1., 0.],
        [1., 2.],
        [0., 0.],
        [2., 0.],
        [2., 2.],
        [0., 1.],
        [2., 1.],
        [1., 0.],
        [0., 0.],
        [1., 2.],
        [0., 0.],
        [1., 2.],
        [1., 0.],
        [0., 0.],
        [2., 2.],
        [1., 2.],
        [2., 2.],
        [2., 0.],
        [0., 0.],
        [1., 2.],
        [0., 2.],
        [2., 2.],
        [0., 0.],
        [2., 0.],
        [1

In [10]:
lambda_0 = np.random.normal(size=(z_dim, k))

In [11]:
lambda_0.shape

(2, 3)

In [12]:
n = np.random.normal(size=(z_dim, k))

In [13]:
n.shape

(2, 3)

In [14]:
a.shape

(100, 2)

In [17]:
lambdas = np.zeros(shape=(z_dim, z_dim, k)) # here, the first z_dim is the number of groups

In [18]:
lambdas.shape

(2, 2, 3)

In [21]:
(np.expand_dims(G[a[0]],-1)).shape

(2, 2, 1)

In [22]:
n.shape

(2, 3)

In [32]:
G[a[0]]

tensor([[1., 0.],
        [0., 1.]])

In [25]:
t = np.expand_dims(G[a[0]],-1) * n[0]

In [26]:
t.shape

(2, 2, 3)

In [31]:
t[:,:,0]

array([[1.41616621, 0.        ],
       [0.        , 1.41616621]])

In [28]:
G

tensor([[1., 0.],
        [0., 1.]])

In [30]:
n[0]

array([1.41616621, 0.55500264, 0.20586224])

In [ ]:
# This is weird. I thought we could get rid of the second d_z and contract it into 1. But I was expecting them to have the same values.

In [25]:
lambdas

array([[[[-1.46722238,  0.72336856,  1.32894517],
         [-1.47673627, -0.04263916,  0.37372134]],

        [[-0.01326652, -0.38974422, -0.21319078],
         [-0.34204375,  0.22373867, -0.03099414]]],


       [[[ 1.36641534, -0.36391423,  0.30015071],
         [ 0.99772384,  0.06838315,  0.39109255]],

        [[ 0.95288259, -2.52470365,  1.63716599],
         [ 1.75858699, -0.56105825,  1.05611458]]],


       [[[ 1.43709595, -1.93034025,  0.85220544],
         [ 0.07326837,  0.04003866, -0.41417918]],

        [[-1.09000431,  0.517375  , -1.37801444],
         [ 0.8531244 ,  0.45127992, -0.72764782]]],


       ...,


       [[[ 0.38455862, -0.87007811, -0.75461628],
         [ 0.213999  , -0.66068732, -0.68987234]],

        [[-0.57976271,  1.00780638, -2.07873712],
         [-0.90562209,  0.55869455, -0.1110131 ]]],


       [[[-0.98467735, -0.79970502, -0.51859711],
         [-2.21174478, -1.46944905, -1.0256234 ]],

        [[-0.10618734, -0.3975162 , -0.2255128 ],
         [

In [26]:
for i in range(len(a)):
    print('i', i)
    for j in range(2):
        print('j', j)
        idx = a[i,j]
        print('idx', idx)
        print(G[idx])
        print('---')

i 0
j 0
idx 0
[1. 0.]
---
j 1
idx 1
[0. 1.]
---
i 1
j 0
idx 1
[0. 1.]
---
j 1
idx 0
[1. 0.]
---
i 2
j 0
idx 0
[1. 0.]
---
j 1
idx 1
[0. 1.]
---
i 3
j 0
idx 0
[1. 0.]
---
j 1
idx 0
[1. 0.]
---
i 4
j 0
idx 0
[1. 0.]
---
j 1
idx 1
[0. 1.]
---
i 5
j 0
idx 0
[1. 0.]
---
j 1
idx 0
[1. 0.]
---
i 6
j 0
idx 0
[1. 0.]
---
j 1
idx 0
[1. 0.]
---
i 7
j 0
idx 0
[1. 0.]
---
j 1
idx 0
[1. 0.]
---
i 8
j 0
idx 0
[1. 0.]
---
j 1
idx 0
[1. 0.]
---
i 9
j 0
idx 1
[0. 1.]
---
j 1
idx 0
[1. 0.]
---


In [27]:
a

array([[0, 1],
       [1, 0],
       [0, 1],
       [0, 0],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0]])

In [28]:
G

array([[1., 0.],
       [0., 1.]])

In [ ]:
for i in range(len(a)):
    print('i', i)
    for j in range(2):
        print('j', j)
        idx = a[i,j]
        print('idx', idx)
        print(G[idx])
        print('---')